# Different Attention Mechanisms in keras 

1. Simple Attention 
2. self Attention
3. Attention with context 
4. Additive Attention 
5. Scaled Attention 
6. MultiHeadAttention


In [2]:
import tensorflow as tf
from keras.layers import * 
from keras.models import *
import numpy as np

Using TensorFlow backend.


# Simple Attention : Bahdanau et a
    
research papper : https://arxiv.org/abs/1409.0473    
    
#### Explaination

input - shape : (batch_size,step_size,lstm_outdim):


output - shape : (batch_size,step_size,1)


W = weight matrix of shape (lstm_outdim X 1)

B = bias vector of shape (step_size X 1) (optional)


#### Steps for calculating attetnion 

Ignoring the batch_size,performing attention using only a dense layer and optional bias 


op1 = input X W     :output (step_size X 1)

op2 = tanh(op1)

op3 = sigmoid(op2)

final_op = op3 * input


# My implementation

In [17]:

# 
class Attention(Layer):
    
    
    def __init__(self):
        super(Attention, self).__init__()
        
        
        
    def build(self,input_shape):
        
        self.W = self.add_weight(
                                shape=(input_shape[-1],1),
                                initializer="he_normal",
                                dtype = "float32",    
                                trainable = True,
                                name = "W"
                                )
        self.B = self.add_weight(
                                shape=(input_shape[1],1),
                                initializer="ones",
                                trainable = True,
                                name = "B"
                                )           
        
        super(Attention, self).build(input_shape)    
        

    def call(self, x,bias = False):

    
        op1 = tf.matmul(x,self.W)
        
        
        
        if bias:
            op1 = tf.add(op1,self.B)
        
        
        op2 = tf.tanh(op1)
    
        op3 = tf.math.softmax(op2)
        
        final_op = x * op3
        
        return tf.reduce_sum(final_op,axis = 1)
    
    
    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()
    
        
x = tf.constant(np.random.random((100,200,300)),dtype = 'float32')
print(x.shape)        
Attention()(x,bias = True).shape  

(100, 200, 300)


TensorShape([100, 300])

# Self attention


Formula a = softmax(Ws2 * tanh ( Ws1 * X.T))

here we introduce ws2 matrix 

Earlier the attention was 1D now we make it 2D
this helps focous on the multile components of a 
sentence 


x - hidden states of 

Here r is the hyperparameter

H = (batch_size,step,dim)
Ws2 = (r,da)
Ws1 = (da,dim)




op1 = Ws1 * H.T : output (da,step)

op2 = tanh(op1) : output (da,step)

op3 = Ws2 * op2 : output (r,step)

softmax is performed along the second dimension 
that along the columnn . look at the shape of 
op3 its (r,batch_size) where r is the hyperparametr
. its the parts being extracted , the parts the network thinks are important. So each column 
holds attention for one input sentence which.
This weights need to sum up to 1. They act as probability saying which word in the sentence is more important.

A = softmax(op3) - will need to look into this 


M = A * H :output (r * step)

M is embedding matrix.
 







In [166]:

# 
class SelfAttention(Layer):
    def __init__(self,batch_size,u,r,da):
        super(SelfAttention, self).__init__()
        
        """
        H = (batch_size,step,dim) or
        H = (batch_size,dim) - selecting this one 
        Ws2 = (r,da) 
        Ws1 = (da,dim)  
        """
        
        self.da = da
        self.r = r
        self.u = u
        init = tf.initializers.he_uniform()
        self.d = u
        
        
        self.Ws1 = tf.Variable(initial_value=init(shape=(self.da, u), dtype="float32"),trainable=True,name="WS1")
        
        self.Ws2 = tf.Variable(initial_value = init(shape = (self.r,self.da),dtype = "float32"),trainable = True,name = "WS2")

    
    def call(self, x):

       

    
        op1 = tf.matmul(self.Ws1,tf.transpose(tf.reshape(x,(-1,self.u))))
        
        
    
        op2 = tf.tanh(op1)
       
        
        op3 = tf.matmul(self.Ws2,op2) # output shape : (30,200)
        
        op3 = tf.reshape(op3,(100,200,-1))
        
         
        
    
        attention_weights = tf.math.softmax(op3,1)
        
    
#         print(attention_weights.shape)

        final_op = tf.matmul(tf.transpose(tf.reduce_sum(attention_weights,0)),tf.transpose(x))
        
        
        #final output should be batch_size X r X U
        
        
        return tf.transpose(final_op),attention_weights
    
    
    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()
    
        
x = tf.constant(np.random.random((100,200,300)),dtype = 'float32')
print(x.shape)        

_,att = SelfAttention(100,300,30,60).call(x) 

(100, 200, 300)
tracking <tf.Variable 'WS1:0' shape=(60, 300) dtype=float32, numpy=
array([[-0.22197285, -0.11108026, -0.06221244, ...,  0.2962583 ,
         0.12028784,  0.03872958],
       [ 0.0748764 ,  0.18606624, -0.10900351, ...,  0.28110692,
        -0.03352141, -0.10248536],
       [-0.09674872, -0.15980174,  0.12203795, ...,  0.0649488 ,
        -0.2368743 , -0.15283988],
       ...,
       [ 0.19190738,  0.04186773,  0.24856421, ...,  0.10236433,
         0.19191793, -0.17187218],
       [ 0.01786709,  0.00816062,  0.1528886 , ...,  0.2369453 ,
        -0.01376432,  0.07767013],
       [ 0.26486418,  0.20375934,  0.00499254, ..., -0.14374104,
         0.19917867, -0.27636725]], dtype=float32)> Ws1
tracking <tf.Variable 'WS2:0' shape=(30, 60) dtype=float32, numpy=
array([[-0.27931112,  0.24099696, -0.35254106, ...,  0.10023117,
        -0.18901902,  0.21257728],
       [ 0.4144494 ,  0.0030632 , -0.33869252, ...,  0.24227178,
         0.22474116, -0.35860145],
       [ 0.17150

In [168]:
sum(att[0,:,2])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0000004>

# Attention With Context 
Paper : Hierarchical Attention Networks for Document Classification

### This Attention gets better score than simple mechanism

In [10]:

#my implementation of Attention with context 

class AttentionContext(Layer):
    
    
    def _init__(self):
        super(AttentionContext,self).__init__()
        
       
        
        
    def build(self,input_shape):
        
        '''
        we need to initialize 2 matrices and 1 bias vector.
        The extra  matrix is the context which is multiplied to the energies 
        
        Here the matrix W will have shape (dim x dim) where dim is the feature dim.
        
        The final output need to remain same as the simple attention to achieve this 
        the vector W is converted into matrix 
        
        The bias will also have different dimensions from the original attention mechanism
        but will will still be a vector 
        
        
        The U vector which is the main change here is called the context vector 
        responsiable for  finding out important information from the energies 
        
        
        '''
        
        self.W = self.add_weight(
                                shape = (input_shape[-1],input_shape[-1]),
                                initializer = "he_normal",
                                dtype = 'float32',
                                trainable = True,
                                name = "W"
                                )
        
        self.B = self.add_weight(
                                shape = (input_shape[-1],),
                                initializer = "ones",
                                dtype = 'float32',
                                trainable = True,
                                name = "B"
                                )
        
        self.U = self.add_weight(
                                shape = (input_shape[-1],),
                                initializer = "he_normal",
                                dtype = 'float32',
                                trainable = True,
                                name = "B"
                                )
        
        
    # I am not implementing mask 
    def call(self,x,bias = True):
            
            
            assert len(x.shape) == 3
            
            energies = K.squeeze(K.dot(x, K.expand_dims(self.W)),axis = -1)  # output (batch_size,steps,dims)
            
#             print(energies.shape)   
            
            if bias:
                
                energies += self.B     # output (batch_size,steps,dims)       
            
            energies = K.tanh(energies)
            
            print(energies.shape) 
            
            context_vector = K.dot(energies,K.expand_dims(self.U))# output (batch_size,steps,1)
            
            
            attention_weights = K.softmax(context_vector,axis = 1)
            
            
            
            
            context = x * attention_weights 

            
            return   K.sum(context,axis = 1)
            
            
            
            
            
            
         

In [11]:
input = tf.constant(np.random.random((100,200,300)),dtype = "float32")            
        
att = AttentionContext() 
att.build(input.shape)
context = att.call(input)

(100, 200, 300)


In [60]:
context.shape

TensorShape([100, 300])

# Using the build function we dont need to specify the input shape

In [62]:
input = tf.constant(np.random.random((190,2000,300)),dtype = "float32")            
AttentionContext()(input)

(190, 2000, 300)


<tf.Tensor 'attention_context_41/Sum:0' shape=(190, 300) dtype=float32>

# Additive attention 

### formula energies e  = V * tanh(W1 * q + W2 * v)


In [91]:

#with attention 

import tensorflow as tf

class BahdanauAttention(tf.keras.layers.Layer):
    
    
      def __init__(self, units):
            super(BahdanauAttention, self).__init__()
            self.units = units
    
    
      def build(self,input_shape):   
            
            print('input_shape',input_shape)
    
            self.W1 = self.add_weight(
                                     shape = (input_shape[-1],self.units),
                                     initializer = 'he_normal',
                                     dtype = 'float32',
                                     trainable = True,
                                     name = "W1"
                                    )
        
            self.W2 = self.add_weight(
                                      shape = (input_shape[-1],self.units),
                                      initializer = 'he_normal',
                                      dtype = 'float32',
                                      trainable = True,
                                      name = "W2"
                                     )
            self.V1 =  self.add_weight(shape = (self.units,1),
                                     initializer = 'he_normal',
                                     dtype = 'float32',
                                     trainable = True,
                                     name = "V"
                                    )
            
            
#             print('w2',self.W2.shape)
                                     

      def call(self, query, values):
    
    
            '''
            value  - (batch_size,steps,dim)
            query  - (batch_size,1,dim) - > expanded query

            dense layer only changes the last layer 

            op1 =  W1 * query  =>  (batch_size,1,units)

            op2 = W2 * values =>  (batch_size,steps,units)  

            ADD =  op1 + op2 during this op1 will be broadcasted to match shape of op2 => (batch_size,steps,units)

            op3 = tanh(ADD)

            energies = V * op3 => (batch_size,steps,1)

            normalizerd_energies/ attention_weights = softmax(energises) => (batch_size,steps,1)

            The reason we normalzie is to maked the dot product of long sequences have similar magnited of dot of small vectors 

            context_vectors = x * attention_weights  =>(batch_size,steps,dims)

            return context_vectors 


            '''
    
    
            assert len(query.shape) == 2
            assert len(values.shape) == 3
    

#             expanded_query = tf.expand_dims(query, axsi = 1)
            
            Q = K.dot(query,self.W1) # output: (batch_size,units)
            
            print('Q:',Q.shape)
            
            V = K.dot(values,self.W2) # output: (batch_size,steps,units)
            
            print('V:',V.shape)
            
               
            ADD = K.expand_dims(Q,axis = 1) + V # output : (batch_size,steps,units)
            
            print('ADD:',ADD.shape)
                
            score = K.dot(ADD,self.V1)
            
            print('Score:',score.shape)

    
            # attention_weights shape == (batch_size, max_length, 1)
            attention_weights = K.softmax(score, axis=1)

            # context_vector shape after sum == (batch_size, hidden_size)
            context_vector = attention_weights * values
            context_vector = K.sum(context_vector, axis=1)

            
            print('context_vector:',context_vector.shape)
            
            return context_vector


values = tf.constant(np.random.normal(0,1,(100,200,300)),dtype = "float32")  
query = tf.constant(np.random.normal(0,1,(100,300)),dtype = "float32") 
activation_layer = BahdanauAttention(10)

activation_layer.build(values.shape)

activation_layer.call(query,values)


input_shape (100, 200, 300)
Q: (100, 10)
V: (100, 200, 10)
ADD: (100, 200, 10)
Score: (100, 200, 1)
context_vector: (100, 300)


<tf.Tensor: shape=(100, 300), dtype=float32, numpy=
array([[-0.10166631, -0.5937331 ,  0.6985316 , ..., -1.2327119 ,
        -2.0880919 ,  0.46112052],
       [ 0.2643503 ,  0.224652  ,  0.72448575, ..., -0.71220255,
         0.9383941 ,  0.44116318],
       [ 0.14608662,  0.32352874, -0.03091997, ..., -0.06793836,
        -0.31154707, -0.10095207],
       ...,
       [-0.22122407,  0.07794187, -0.26526845, ..., -0.6523165 ,
         0.4636118 ,  0.35176542],
       [ 0.0133392 , -0.1013723 , -0.03963846, ..., -0.3964976 ,
         0.08073253, -0.10223395],
       [ 0.1357487 ,  0.19757317,  0.07662726, ..., -0.11371756,
         0.18536097, -0.5483618 ]], dtype=float32)>

# Scaled dot product attention



We have 3 input vectors queries , keys and values. Queries and keys have same dimenstions 
We require 3 matrices Wq,Wk,Wq.

Q = Wq * q 
K = Wk * k
V = Wv * v
attention =  sigmoid((Q * K) / sqrt(d)) * V

In [55]:
class scaled_dot_product_attention(Layer):
    
    
    def __init__(self,dim):
        
        super(scaled_dot_product_attention,self).__init__()
        
        self.dim = dim
        
    def build(self,input_shape):
        
        '''
        
        input_shape = (batch_size,steps,dims)
        
        steps = dmodel
        
        dims = dk
        
        matirx Wq,Wk,Wv
        
        shape of all is same dmodel x dk for Wq and Wk
        and dmodel x dv for Wv
        
        we take dk == dv but they can be different.
        
        '''
        
        self.Wq = self.add_weight( 
                                  shape = (input_shape[-1],self.dim),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wq"  
                                 ) 
        
        self.Wk = self.add_weight( 
                                  shape = (input_shape[-1],self.dim),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wk"  
                                 ) 
        
        
        self.Wv = self.add_weight( 
                                  shape = (input_shape[-1],self.dim),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wv"  
                                 ) 
        
        
        print(self.Wv.shape)
    
        
    def call(self,query,key,value):
        
        '''
        For classification tast query == value == key 
        key and value can be the same input.
        
        query shape = (batch_size,steps,dims)
        '''
        
        input_shape = query.shape
        
        Q = tf.matmul(query,self.Wq) #output: (batch_size,steps,step)
        
        K = tf.matmul(query,self.Wk) #output: (batch_size,steps,step)
        
        V = tf.matmul(query,self.Wv) #output: (batch_size,steps,step)
        
        t1 =tf.divide(tf.matmul(Q,K,transpose_b = True),np.sqrt(input_shape[-1]))
        
        t1 = tf.math.softmax(t1,axis = 1)
        
        attention  = tf.matmul(t1,V)
        
        
        print(attention.shape)
        
        
x = tf.constant(np.random.normal(0,1,(100,200,300)),dtype = "float32")  
# query = tf.constant(np.random.normal(0,1,(100,300)),dtype = "float32") 
activation_layer = scaled_dot_product_attention(64)

activation_layer.build(x.shape)

activation_layer.call(x,x,x)        
        
        

(300, 64)
(100, 200, 64)


In [19]:
class MultiHeadAttention(Layer):
    
    
    def __init__(self,head,dim):
        
        super(MultiHeadAttention,self).__init__()
        
        self.dim = dim
        self.head = head
        
    def build(self,input_shape):
        
        '''
        
        input_shape = (batch_size,steps,dims)
        
        dims = dmodel
        
        dk = units 
        
        matirx Wq,Wk,Wv
        
        shape of all is same dmodel x dk for Wq and Wk
        and dmodel x dv for Wv
        
        we take dk == dv but they can be different.
        
        
        output of the embedding 100,75,300
        
        '''
        
        self.Wq = self.add_weight( 
                                  shape = (input_shape[0][-1],self.dim * self.head),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wq"  
                                 ) 
        
        self.Wk = self.add_weight( 
                                  shape = (input_shape[1][-1],self.dim * self.head),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wk"  
                                 ) 
        
        
        self.Wv = self.add_weight( 
                                  shape = (input_shape[2][-1],self.dim * self.head),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wv"  
                                 ) 
        
        self.Wo = self.add_weight( 
                                  shape = (self.head* self.dim,input_shape[0][-1]),
                                  initializer = "he_normal",
                                  dtype = "float32",
                                  trainable = True,
                                  name = "Wo"  
                                 )  
        
#         print(self.Wv.shape)

        super(MultiHeadAttention,self).build(input_shape) 
    
        
    def call(self,x):
        
        '''
        For classification tast query == value == key 
        key and value can be the same input.
        
        query shape = (batch_size,steps,dims)
        '''
        
        query,key,value = x
        
        Q = tf.matmul(query,self.Wq) #output: (batch_size,steps,step)
        
        K = tf.matmul(query,self.Wk) #output: (batch_size,steps,step)
        
        V = tf.matmul(query,self.Wv) #output: (batch_size,steps,step)
        
        t1 =tf.divide(tf.matmul(Q,K,transpose_b = True),np.sqrt(self.dim))
        
        t1 = tf.math.softmax(t1,axis = 1)
        
        head  = tf.matmul(t1,V)
        
        context_vector = tf.matmul(head,self.Wo)
        
        
#         print(head.shape)
#         print(self.Wo.shape)

        print(context_vector.shape)
        
        return context_vector
    
x = tf.constant(np.random.normal(0,1,(100,200,300)),dtype = "float32")  
# query = tf.constant(np.random.normal(0,1,(100,300)),dtype = "float32") 
# activation_layer = MultiHeadAttention(3,64)

# activation_layer.build(x.shape)

# activation_layer.call(x,x,x)        
        
MultiHeadAttention(3,64)([x,x,x])
        

(100, 200, 300)


<tf.Tensor 'multi_head_attention_2/MatMul_5:0' shape=(100, 200, 300) dtype=float32>